In [14]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from operator import itemgetter
from langchain_core.runnables import RunnableMap
from langchain import LLMChain, PromptTemplate  # if needed for other functionalities
from langchain_core.output_parsers import StrOutputParser

from dotenv import load_dotenv
import os

In [15]:
# Load API key
_ = load_dotenv()

# Trial 1: only run chain A

In [10]:
# 1) System Message
system_prompt = """You are an extremely creative entrepreneur who has successfully designed and built many commercially successful products.
You are helping me generate profitable product ideas.
"""
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)

# 2) Chain A - generate 5 questions
user_prompt_a = HumanMessagePromptTemplate.from_template(
    """Please help me come up with product ideas about {product}, targeting {target_customers}.
    I want the ideas to be BOLD and INNOVATIVE.

    First, generate exactly 5 thought-provoking questions that force me to explore new angles or possibilities for {product} and {target_customers}.
    Please output them in the following numbered list format:
    1. [keyword(s)]: [question]
    2. [keyword(s)]: [question]
    3. ...
    4. ...
    5. ...
    """
)

chat_prompt = ChatPromptTemplate.from_messages([
    system_message_template,
    user_prompt_a
])

# 3) Create the chatmodel 
llm = ChatOpenAI(
    temperature=0.8,
)

# 4) Pipe the prompt into the LLM to form a runnable pipeline
pipeline_a = chat_prompt | llm

In [12]:
response = pipeline_a.invoke({"product": "a notebook", "target_customers": "ux designers"})

# print (response.content)

1. Customization: How can a notebook be designed to adapt and cater to the diverse UX design process, allowing for personalized layouts and templates?
2. Collaboration: In what ways can a notebook facilitate seamless collaboration among UX designers, whether in person or remotely?
3. Integration: How can a notebook integrate digital tools and platforms to enhance the UX design workflow, bridging the gap between analog and digital processes?
4. Sustainability: Can a notebook be created with sustainable materials and eco-friendly practices, aligning with the values of environmentally conscious UX designers?
5. Augmented Reality: How might incorporating augmented reality features into a notebook revolutionize the way UX designers ideate, prototype, and present their work?


# Trial 2: combine Chain A and Chain B

In [23]:
# 1) System Message
system_prompt = """You are an extremely creative entrepreneur who has successfully designed and built many commercially successful products.
You are helping me generate profitable product ideas.
"""
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)

# 2) Chain A - generate 5 questions
user_prompt_a = HumanMessagePromptTemplate.from_template(
    """Please help me come up with product ideas about {product}, targeting {target_customers}.
    I want the ideas to be BOLD and INNOVATIVE.

    First, generate exactly 5 thought-provoking questions that force me to explore new angles or possibilities for {product} and {target_customers}.
    Please output them in the following numbered list format:
    1. [keyword(s)]: [question]
    2. [keyword(s)]: [question]
    3. ...
    4. ...
    5. ...
    """
)

chat_prompt_a = ChatPromptTemplate.from_messages([
    system_message_template,
    user_prompt_a
])

# 3) Chain B - generate 3 short answers to each question
user_prompt_b = HumanMessagePromptTemplate.from_template(
    """We have the following 5 questions about {product} targeting {target_customers}:

{questions}

Now, for each question, generate 3 potential directions (each under 50 words) that are inspirational 
to guide users in developing their own creative solutions.

Format them immediately after each question using an unordered list, for example:

1. [keyword(s)]: [question]
- direction 1
- direction 2
- direction 3

2. [keyword(s)]: [question]
- direction 1
- direction 2
- direction 3

... and so on.

Remember to keep each direction under 50 words.
"""
)
chat_prompt_b = ChatPromptTemplate.from_messages([
    system_message_template, 
    user_prompt_b
])

# 4) Create a ChatOpenAI model
llm = ChatOpenAI(
    temperature=0.8,
)

# 5) Build a single pipeline for both steps
pipeline = (
    RunnableMap({
        "questions": chat_prompt_a | llm,  # Chain A
        "product": itemgetter("product"),
        "target_customers": itemgetter("target_customers"),
    })
    | chat_prompt_b
    | llm  # Chain B
)


# 6) Use the pipeline

if __name__ == "__main__":
    inputs = {
        "product": "a note taker",
        "target_customers": "college students"
    }

    result = pipeline.invoke(inputs)

    # 'result' is a ChatMessage object; get its text with .content
    print(result.content)

1. Mobility & Integration: How can a note taker seamlessly integrate with a student's existing digital devices and platforms to ensure notes are easily accessible on the go?
- Create a cloud-based note-taking app that syncs across all devices instantly.
- Develop a smart pen that transfers handwritten notes to digital formats automatically.
- Implement NFC technology for easy sharing and access on multiple devices.

2. Artificial Intelligence: How can AI be leveraged in a note taker to provide personalized study recommendations based on a student's note-taking habits and learning style?
- Use AI to analyze notes and suggest relevant study materials or resources.
- Implement AI chatbots for answering queries related to notes and study topics.
- Create an AI-powered study planner based on note-taking patterns.

3. Collaboration & Sharing: In what ways can a note taker facilitate real-time collaboration and sharing features to enhance group study sessions among college students?
- Enable 